In [1]:
# Import libraries
import requests
import pandas as pd
from bokeh.plotting import output_file, show
from bokeh.models import Select, Div, Button, ColumnDataSource, HTMLTemplateFormatter,TableColumn, DataTable
from bokeh.models.widgets import DateRangePicker, DataTable, TableColumn
from bokeh.layouts import column
# Import necessary modules
from ipywidgets import interact, widgets, VBox, HBox
from IPython.display import display, HTML
from bokeh.io import push_notebook, show, output_notebook
import pandas as pd
# Import the necessary modules
from ipywidgets import HTML
from IPython.display import display, HTML as IPHTML

In [2]:
# Import all APIs form config.py
# WeatherAPI key
weather_key = "11da115d9a8d4556bd993522240506"

In [3]:
def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials",
        "client_id": 'gAJOLRiyFCjYVuCEIspShbBroNVLSgpw',
        "client_secret": 'ZDHiCX0h0NYIBm0B'
    }
 
    response = requests.post(url, headers=headers, data=data)
    if response.status_code == 200:
        # Extract the access token from the response JSON
        access_token = response.json()['access_token']
        print("Access Token:", access_token)
    else:
        # Print error message if the request was not successful
        print("Failed to obtain access token:", response.status_code, response.text)
    return access_token

# Get access token
access_token = get_access_token()

Access Token: enCXItQPHBeMT7H1FOgj4JYzADGf


## Get flights according to city, airport and departure date

In [4]:
# List of cities
cities = ['Madrid', 'London', 'Paris', 'Barcelona', 'Athens']
# Function to get flight details
def get_flight_details(access_token, originLocationCode, destinationLocationCode, departureDate):
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    headers = {
        "Authorization": f'Bearer {access_token}',
        "Content-Type": "application/json"
    }
    params = {
        "originLocationCode": originLocationCode,
        "destinationLocationCode": destinationLocationCode,
        "departureDate": departureDate,
        "adults": 1,
        "max": 3
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to get flight details:", response.status_code, response.text)
        return None

In [5]:

# Function to process flight data
def process_flight_data(flight_offers_response):
    if flight_offers_response:
        # Extract flight details and create DataFrame
        flight_data = []
        for flight in flight_offers_response['data']:
            itinerary = flight['itineraries'][0]
            segment = itinerary['segments'][0]
            departure_airport = segment['departure']['iataCode']
            arrival_airport = segment['arrival']['iataCode']
            departure_time = segment['departure']['at'][-8:-3]
            arrival_time = segment['arrival']['at'][-8:-3]

            # Convert string time to hours and minutes
            departure_hour, departure_minute = map(int, departure_time.split(':'))
            arrival_hour, arrival_minute = map(int, arrival_time.split(':'))

            # Calculate duration in minutes
            duration_minutes = (arrival_hour * 60 + arrival_minute) - (departure_hour * 60 + departure_minute)

            # Convert duration to hours and minutes
            duration_hours = duration_minutes // 60
            duration_minutes = duration_minutes % 60

            duration_str = f"{duration_hours:02d}:{duration_minutes:02d}"

            price = float(flight['price']['total'])
            currency = flight['price']['currency']
            price_with_currency = f"{price} {currency}"

            # Combine price with currency
            flight_data.append([departure_airport, arrival_airport, departure_time, arrival_time, duration_str, price_with_currency])

        # Creating DataFrame
        df = pd.DataFrame(flight_data, columns=['DepartureAirport', 'ArrivalAirport', 'DepartureTime', 'ArrivalTime', 'Duration(hours)', 'PriceWithCurrency'])
        return df
    else:
        print("No flight offers data available.")
        return None

In [6]:
# Function to get airport codes based on city
def get_airport_codes(city):
    api_url = f'https://api.api-ninjas.com/v1/airports?city={city}'
    headers = {'X-Api-Key': 'nrc3S/XZWbfao3LeySY9hg==I1u1i9JSOP6NSPFw'}
    response = requests.get(api_url, headers=headers)
    if response.status_code == requests.codes.ok:
        airport_data = response.json()
        airport_dict = {}
        for airport in airport_data:
            airport_code = airport.get('iata')
            if airport_code:
                airport_name = airport.get('name')
                airport_dict[airport_name] = airport_code
        return airport_dict
    else:
        print("Error:", response.status_code, response.text)
        return {}

In [7]:
'''# Function to handle city selection change
def update_selected_city(attr, old, new):
    # Update the airport select options based on the new city
    airport_codes = get_airport_codes(new)
    airport_select.options = airport_codes
    airport_select.value = airport_codes[0] if airport_codes else ''

    result_div.text = f"Selected city: {new}"'''

'# Function to handle city selection change\ndef update_selected_city(attr, old, new):\n    # Update the airport select options based on the new city\n    airport_codes = get_airport_codes(new)\n    airport_select.options = airport_codes\n    airport_select.value = airport_codes[0] if airport_codes else \'\'\n\n    result_div.text = f"Selected city: {new}"'

In [8]:
'''# Get initial airport codes
initial_airport_codes = get_airport_codes(cities[0])
airport_select = Select(title="Airport:", value=initial_airport_codes[0] if initial_airport_codes else '', options=initial_airport_codes)
'''

'# Get initial airport codes\ninitial_airport_codes = get_airport_codes(cities[0])\nairport_select = Select(title="Airport:", value=initial_airport_codes[0] if initial_airport_codes else \'\', options=initial_airport_codes)\n'

In [9]:
# Function to update airport options based on city selection
def update_airport_options(change):
    city = change['new']
    airports = get_airport_codes(city)
    if airports:
        airport_select.options = airports
        airport_select.value = list(airports.values())[0]

In [10]:
# Define the function to update flight offers
def update_flight_offers(city, access_token, originLocationCode, destinationLocationCode, departure_date):
    # Print values for debugging
    print("City:", city)
    print("Departure Date:", departure_date)
    print("Origin Location Code:", originLocationCode)
    print("Destination Location Code:", destinationLocationCode)
    
    # Call the function to get flight details
    flight_offers_response = get_flight_details(access_token, originLocationCode, destinationLocationCode, departure_date)
    print(flight_offers_response)
    
    # Process flight data
    if flight_offers_response:
        flight_data_table = process_flight_data(flight_offers_response)
        if flight_data_table is not None:
            # Convert DataFrame to HTML table
            flight_details_html = flight_data_table.to_html(index=False)
            # Display the HTML table
            flight_details_output.value = flight_details_html
    else:
        print("No flight offers found.")

In [12]:
# Import the necessary modules
from ipywidgets import HTML
from IPython.display import display, HTML as IPHTML

city_select = widgets.Dropdown(options=cities, description='Select City')
airport_select = widgets.Dropdown(description='Select Airport')
departure_date_picker = widgets.DatePicker(description='Select Date')
update_button = widgets.Button(description='Get Flights')
flight_details_output = widgets.Output()

# Define the function to update flight offers
def update_flight_offers(city, access_token, originLocationCode, destinationLocationCode, departure_date):
    # Print values for debugging
    print("City:", city)
    print("Departure Date:", departure_date)
    print("Origin Location Code:", originLocationCode)
    print("Destination Location Code:", destinationLocationCode)
    
    # Call the function to get flight details
    flight_offers_response = get_flight_details(access_token, originLocationCode, destinationLocationCode, departure_date)
    print(flight_offers_response)
    
    # Process flight data
    if flight_offers_response:
        flight_data_table = process_flight_data(flight_offers_response)
        if flight_data_table is not None:
            # Convert DataFrame to HTML table
            flight_details_html = flight_data_table.to_html(index=False)
            # Display the HTML table
            flight_details_output.value = flight_details_html
    else:
        print("No flight offers found.")

# Create the HTML widget for displaying flight details
flight_details_output = HTML()

# Function to handle update button click
def on_update_button_clicked(button):
    city = city_select.value
    access_token = get_access_token()
    originLocationCode = 'MAD'
    destinationLocationCode = airport_select.value
    departure_date = departure_date_picker.value.strftime('%Y-%m-%d')
    
    update_flight_offers(city, access_token, originLocationCode, destinationLocationCode, departure_date)

city_select.observe(update_airport_options, names='value')

update_button.on_click(on_update_button_clicked)

# Define a container for the widgets
inputs = VBox([city_select, airport_select, departure_date_picker, update_button])

# Display the widgets
display(inputs, flight_details_output)


HTML(value='')

# ONLY UNTIL HERE - DO NOOOOT RUN MORE

# STOP RUNNING THE CODE 

# ENOUGH ALREADY

In [201]:
# Function to fetch weather data for the next 10 days for a single city
def fetch_10_day_forecast(city, api_key):
    base_url = 'http://api.weatherapi.com/v1'
    url = f"{base_url}/forecast.json?key={api_key}&q={city}&days=10"
    response = requests.get(url)
    data = response.json()
    
    # Check if the response contains the necessary forecast data
    if 'forecast' in data and 'forecastday' in data['forecast']:
        forecast_data = data['forecast']['forecastday']
        extracted_data = []
        
        for day in forecast_data:
            date = day['date']
            temp = day['day']['avgtemp_c']
            humidity = day['day']['avghumidity']
            cloudiness = day['day']['condition']['text']
            wind_speed = day['day']['maxwind_kph']
            
            extracted_data.append({
                'city': city,
                'date': date,
                'temp': temp,
                'humidity': humidity,
                'condition': cloudiness,
                'wind_speed': wind_speed
            })
        
        return pd.DataFrame(extracted_data)
    else:
        print(f"No forecast data available for {city}")
        return pd.DataFrame()

Comment on what the function does and the input/output

In [170]:
# Function to find the best 7 consecutive days based on specific filters
def find_best_7_days(forecast_df, criteria='temp'):
    best_city = None
    best_start_date = None
    best_average = float('-inf') if criteria == 'temp' else float('inf')
    
    for city in forecast_df['city'].unique():
        city_df = forecast_df[forecast_df['city'] == city]
        
        # Ensure the dates are sorted
        city_df = city_df.sort_values(by='date')
        
        # Iterate over possible 7-day windows
        for i in range(len(city_df) - 6):
            window = city_df.iloc[i:i+7]
            
            if criteria == 'temp':
                average = window['temp'].mean()
                if average > best_average:
                    best_average = average
                    best_city = city
                    best_start_date = window.iloc[0]['date']
            elif criteria == 'humidity':
                average = window['humidity'].mean()
                if average < best_average:
                    best_average = average
                    best_city = city
                    best_start_date = window.iloc[0]['date']
            elif criteria == 'wind_speed':
                average = window['wind_speed'].mean()
                if average < best_average:
                    best_average = average
                    best_city = city
                    best_start_date = window.iloc[0]['date']
    
    return best_city, best_start_date, best_average

In [171]:
# Function to handle button click to update the recommended dates
def update_recommended_dates():
    selected_city = city_select.value
    start_date, end_date = date_range_picker.value_as_datetime
    
    filtered_df = all_forecast_df[(all_forecast_df['city'] == selected_city) & 
                                  (all_forecast_df['date'] >= start_date) & 
                                  (all_forecast_df['date'] <= end_date)]
    
    if not filtered_df.empty:
        avg_temp = filtered_df['temp'].mean()
        avg_humidity = filtered_df['humidity'].mean()
        result_div.text = f"Avg. temperature: {avg_temp:.2f}°C<br>Avg. humidity: {avg_humidity:.2f}%"
    else:
        result_div.text = f"No data available for {selected_city} in the selected date range."


In [172]:
# Function to handle date range picker change
def show_selected_date_range(attr, old, new):
    result_div.text = f"Selected city: {city_select.value}<br>Selected date range: {new[0]} to {new[1]}"

In [202]:



# Initialize an empty DataFrame to store the combined forecast data
all_forecast_df = pd.DataFrame()

# Loop through each city and fetch the forecast data
for city in cities:
    city_forecast_df = fetch_10_day_forecast(city, weather_key)
    all_forecast_df = pd.concat([all_forecast_df, city_forecast_df], ignore_index=True)

# Create Select for city input with the cities from the DataFrame
cities = all_forecast_df['city'].unique().tolist()
city_select = Select(title="City:", value=cities[0], options=cities)

# Function to apply color based on temperature
def apply_temp_color(temp):
    if temp < 20:
        return 'mediumaquamarine'
    elif 20 <= temp < 30:
        return 'lightsalmon'
    else:
        return 'brown'



In [174]:

# Function to apply color based on condition - this is not a function, its a dictionary, make it a global variable (?)
condition_mapper = {
    'Sunny': 'gold',
    'Partly cloudy': 'lightblue',
    'Cloudy': 'gray',
    'Rain': 'blue',
    'Clear': 'white'
}

#this is the function
def apply_condition_color(condition):
    return condition_mapper.get(condition, 'white')

In [175]:
# Add columns for coloring
all_forecast_df['temp_color'] = all_forecast_df['temp'].apply(apply_temp_color)
all_forecast_df['condition_color'] = all_forecast_df['condition'].apply(apply_condition_color)

# Why are we outputing an html file? is it to create the web page?
# Output to a new HTML file
output_file("index.html")

In [176]:
# Find the best 7 consecutive days for high temperature, low humidity, and low wind speed
best_city_temp, best_start_date_temp, best_average_temp = find_best_7_days(all_forecast_df, 'temp')
best_city_humidity, best_start_date_humidity, best_average_humidity = find_best_7_days(all_forecast_df, 'humidity')
best_city_wind_speed, best_start_date_wind_speed, best_average_wind_speed = find_best_7_days(all_forecast_df, 'wind_speed')


In [177]:
# Format best_average_temp, best_average_humidity, and best_average_wind_speed to 1 decimal place
best_average_temp_formatted = f"{best_average_temp:.1f}"
best_average_humidity_formatted = f"{best_average_humidity:.1f}"
best_average_wind_speed_formatted = f"{best_average_wind_speed:.1f}"


In [178]:
# Create a Div to show recommended dates
recommended_dates_temp = Div(text=f"<b>Best week based on high temperature:</b> {best_city_temp}, starting from {best_start_date_temp} with average temp {best_average_temp_formatted}°C.<br>",
                             width=800, height=50)

recommended_dates_humidity = Div(text=f"<b>Best week based on low humidity:</b> {best_city_humidity}, starting from {best_start_date_humidity} with average humidity {best_average_humidity_formatted}%.<br>",
                                 width=800, height=50)

recommended_dates_wind_speed = Div(text=f"<b>Best week based on low wind speed:</b> {best_city_wind_speed}, starting from {best_start_date_wind_speed} with average wind speed {best_average_wind_speed_formatted} kph.<br>",
                                  width=800, height=50)


In [179]:
# Logo and header
logo_image = Div(text="<img src='sunnylogo.png' style='width:100px;height:100px;'>")
title_div = Div(text="<h1 style='font-family: Cooper Black, sans-serif; font-size: 36px; color: teal;'><center>Holidays by Sunny Side Up</center></h1>",
                sizing_mode='stretch_width')

# Create a Button to update the recommended dates and table
update_button = Button(label="Discover this city", css_classes=['custom-button'])


# Create a DateRangePicker widget
date_range_picker = DateRangePicker(title="Select a date range:")

# Create a Div element for displaying the selected city and date range
result_div = Div()

In [180]:
# Create a DataTable to display the forecast data
columns = [
    TableColumn(field='city', title='City'),
    TableColumn(field='date', title='Date'),
    TableColumn(field='temp', title='Temperature', width=100, formatter=HTMLTemplateFormatter(template='<div style="background:<%= temp_color %>; color: black;"> <%= value %> </div>')),
    TableColumn(field='humidity', title='Humidity', width=100),
    TableColumn(field='condition', title='Condition', width=100, formatter=HTMLTemplateFormatter(template='<div style="background:<%= condition_color %>; color: black;"> <%= value %> </div>')),
    TableColumn(field='wind_speed', title='Wind Speed', width=100),
]

In [181]:
source = ColumnDataSource(all_forecast_df)
data_table = DataTable(source=source, columns=columns, width=800, height=280)

In [26]:
#flight_offers_response = get_flight_details(access_token, originLocationCode, destinationLocationCode, departureDate)


In [28]:
#df = process_flight_data(flight_offers_response)

In [29]:
'''flight_source = ColumnDataSource(df)
flight_columns = [
    TableColumn(field='DepartureAirport', title='Departure Airport'),
    TableColumn(field='ArrivalAirport', title='Arrival Airport'),
    TableColumn(field='DepartureTime', title='Departure Time'),
    TableColumn(field='ArrivalTime', title='Arrival Time'),
    TableColumn(field='Duration(hours)', title='Duration (hours)'),
    TableColumn(field='PriceWithCurrency', title='Price')
]
'''

In [30]:
#flight_data_table = DataTable(source=flight_source, columns=flight_columns, width=800, height=280)

In [31]:
'''# Attach the functions to the button click events
update_button.on_click(update_recommended_dates)
date_range_picker.on_change('value', show_selected_date_range)
originLocationCode = city_select.on_change('value', update_selected_city)'''


In [158]:
# Layout the widgets with city input, button, and result div
inputs = column(city_select, airport_select, date_range_picker, update_button)
header = column(logo_image, title_div)
results = column(recommended_dates_temp, recommended_dates_humidity, recommended_dates_wind_speed, result_div, data_table,flight_data_table)
layout = column(header, inputs, results, sizing_mode='stretch_both', css_classes=['app-layout'])

ValueError: Only LayoutDOM items can be inserted into a column. Tried to insert: Dropdown(description='Select City', index=1, options=('Madrid', 'London', 'Paris', 'Barcelona'), value='London') of type <class 'ipywidgets.widgets.widget_selection.Dropdown'>

In [33]:
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [185]:
from bokeh.models import ColumnDataSource, DataTable, TableColumn
from bokeh.layouts import column

import requests
import pandas as pd
from ipywidgets import VBox

# Function to handle update button click
def on_update_button_clicked(button):
    city = city_select.value
    access_token = get_access_token()  # Make sure to define get_access_token function
    originLocationCode = 'MAD'  # Assuming 'MAD' is the origin airport code
    destinationLocationCode = airport_select.value
    departure_date = departure_date_picker.value.strftime('%Y-%m-%d')
    
    # Get flight details
    flight_offers_response = get_flight_details(city, access_token, originLocationCode, destinationLocationCode, departure_date)
    
    if flight_offers_response:
        # Process flight data and update flight_source
        flight_data = process_flight_data(flight_offers_response)
        flight_source.data = flight_data

# Define a container for the widgets
inputs = column(city_select, airport_select, departure_date_picker, update_button)

# Display the widgets
display(inputs, flight_details_output)

# Define the function to get flight details
def get_flight_details(city, access_token, originLocationCode, destinationLocationCode, departure_date):
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    headers = {
        "Authorization": f'Bearer {access_token}',
        "Content-Type": "application/json"
    }
    params = {
        "originLocationCode": originLocationCode,
        "destinationLocationCode": destinationLocationCode,
        "departureDate": departure_date,
        "adults": 1,
        "max": 3
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to get flight details:", response.status_code, response.text)
        return None

# Define the function to process flight data
def process_flight_data(flight_offers_response):
    flight_data = {
        'DepartureAirport': [],
        'ArrivalAirport': [],
        'DepartureTime': [],
        'ArrivalTime': [],
        'Duration(hours)': [],
        'PriceWithCurrency': []
    }

    for flight in flight_offers_response['data']:
        itinerary = flight['itineraries'][0]
        segment = itinerary['segments'][0]
        departure_airport = segment['departure']['iataCode']
        arrival_airport = segment['arrival']['iataCode']
        departure_time = segment['departure']['at'][-8:-3]
        arrival_time = segment['arrival']['at'][-8:-3]

        # Calculate duration in hours
        departure_hour, departure_minute = map(int, departure_time.split(':'))
        arrival_hour, arrival_minute = map(int, arrival_time.split(':'))
        duration_hours = arrival_hour - departure_hour

        price = float(flight['price']['total'])
        currency = flight['price']['currency']
        price_with_currency = f"{price} {currency}"

        flight_data['DepartureAirport'].append(departure_airport)
        flight_data['ArrivalAirport'].append(arrival_airport)
        flight_data['DepartureTime'].append(departure_time)
        flight_data['ArrivalTime'].append(arrival_time)
        flight_data['Duration(hours)'].append(duration_hours)
        flight_data['PriceWithCurrency'].append(price_with_currency)

    return flight_data

# Create a ColumnDataSource for flight data
flight_source = ColumnDataSource(data={})

# Create DataTable columns
flight_columns = [
    TableColumn(field='DepartureAirport', title='Departure Airport'),
    TableColumn(field='ArrivalAirport', title='Arrival Airport'),
    TableColumn(field='DepartureTime', title='Departure Time'),
    TableColumn(field='ArrivalTime', title='Arrival Time'),
    TableColumn(field='Duration(hours)', title='Duration (hours)'),
    TableColumn(field='PriceWithCurrency', title='Price')
]

# Create DataTable for flight data
flight_data_table = DataTable(source=flight_source, columns=flight_columns, width=800, height=280)


ValueError: Only LayoutDOM items can be inserted into a column. Tried to insert: Dropdown(description='Select Airport', index=6, options={'London Airport': 'YXU', 'London Luton Airport': 'LTN', 'London Biggin Hill Airport': 'BQH', 'London Gatwick Airport': 'LGW', 'London City Airport': 'LCY', 'London Heathrow Airport': 'LHR', 'London Stansted Airport': 'STN', 'RAF Northolt': 'NHT'}, value='STN') of type <class 'ipywidgets.widgets.widget_selection.Dropdown'>